In [65]:
import os
import unittest
import time
from pymatgen.core.structure import Molecule
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import OpenBabelNN
from pymatgen.util.testing import PymatgenTest
from pymatgen.analysis.reaction_network.reaction_network import ReactionNetwork
from pymatgen.analysis.fragmenter import Fragmenter, metal_edge_extender
from pymatgen.entries.mol_entry import MoleculeEntry
from monty.serialization import loadfn
import openbabel as ob
import networkx as nx

In [66]:
from crystal_toolkit.helpers.pythreejs_renderer import view_old

In [3]:
reextended_entries = []
entries = loadfn("../../../../test_files/reaction_network_files/LiEC_reextended_entries.json")
for entry in entries:
    if "optimized_molecule" in entry["output"]:
        mol = entry["output"]["optimized_molecule"]
    else:
        mol = entry["output"]["initial_molecule"]
    H = float(entry["output"]["enthalpy"])
    S = float(entry["output"]["entropy"])
    E = float(entry["output"]["final_energy"])
    mol_entry = MoleculeEntry(molecule=mol,energy=E,enthalpy=H,entropy=S,entry_id=entry["task_id"])
    if mol_entry.formula == "Li1":
        if mol_entry.charge == 1:
            reextended_entries.append(mol_entry)
    else:
        reextended_entries.append(mol_entry)

In [4]:
RN = ReactionNetwork(reextended_entries)

682 input entries
677 connected entries
569 unique entries
Missing H3O+, will not add concerted water splitting rxn1
Adding concerted water splitting rxn2: 2H2O + 2e- -> H2 + 2OH-
Water rxn2 free energy = 0.9488610461256028


In [5]:
EC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/EC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
EC_mg = metal_edge_extender(EC_mg)

LiEC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/LiEC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
LiEC_mg = metal_edge_extender(LiEC_mg)

LEDC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/LEDC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
LEDC_mg = metal_edge_extender(LEDC_mg)

LEMC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/LEMC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
LEMC_mg = metal_edge_extender(LEMC_mg)

EC_ind = None
LiEC_ind = None
LEDC_ind = None
LEMC_ind = None
for entry in RN.entries["C3 H4 O3"][10][0]:
    if EC_mg.isomorphic_to(entry.mol_graph):
        EC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C3 H4 Li1 O3"][12][1]:
    if LiEC_mg.isomorphic_to(entry.mol_graph):
        LiEC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C4 H4 Li2 O6"][17][0]:
    if LEDC_mg.isomorphic_to(entry.mol_graph):
        LEDC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C3 H5 Li1 O4"][13][0]:
    if LEMC_mg.isomorphic_to(entry.mol_graph):
        LEMC_ind = entry.parameters["ind"]
        break
Li1_ind = RN.entries["Li1"][0][1][0].parameters["ind"]

print("EC_ind",EC_ind)
print("LiEC_ind",LiEC_ind)
print("LEDC_ind",LEDC_ind)
print("LEMC_ind",LEMC_ind)
print("Li1_ind",Li1_ind)

EC_ind 456
LiEC_ind 424
LEDC_ind 511
LEMC_ind 469
Li1_ind 556


In [70]:
ind=556
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-5.329070518200751e-16, left=-5.329070518200751e…

1


-201.49170734039296

In [64]:
ind=41
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind]

Renderer(background='white', camera=OrthographicCamera(bottom=-3.04935337968, left=-3.04935337968, near=-2000.…

0


MoleculeEntry 115878 - C1 Li1 O3 - E5 - C0
Energy = -271.3720 Hartree
Correction = 0.0000 Hartree
Enthalpy = 13.2350 kcal/mol
Entropy = 70.0740 cal/mol.K
Free Energy = -7384.7413 eV
Parameters:
ind = 41

In [49]:
for neighbor in list(RN.graph.neighbors(420)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

420,419
rxn_type : One electron reduction
bipartite : 1
energy : -0.10673717229497015
free_energy : -0.7448082500937745

420,423
rxn_type : Intramolecular single bond formation
bipartite : 1
energy : 0.0377282729020294
free_energy : 1.2093853860187664

420,354+544
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.3561941055910438
free_energy : 9.406364191265311

420,17+112
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.21171106493721936
free_energy : 5.017833742112998

420,18+110
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.18166385452042277
free_energy : 4.216097982036445

420,19+108
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.32811420511234246
free_energy : 8.197687310999754

420,355+544
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.33625550868202936
free_energy : 8.85741642711946

42

In [13]:
for neighbor in list(RN.graph.neighbors(474)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

474,473
rxn_type : One electron reduction
bipartite : 1
energy : -0.08791600709999958
free_energy : -0.2423167564408346

474+PR_11,7
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.06301589485099957
free_energy : -1.3908328395788203

474+PR_12,8
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.018394655645999514
free_energy : -0.2105903542338865

474+PR_21,19
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.0809593017609922
free_energy : -1.866753251131854

474+PR_23,20
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.03034432508300089
free_energy : -0.49996023164806047

474+PR_37,28
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.054634213966999745
free_energy : -1.1508368558471602

474+PR_38,29
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.004152531853997665
free_energy : 

In [15]:
for neighbor in list(RN.graph.neighbors(35)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

35,36
rxn_type : One electron oxidation
bipartite : 1
energy : 0.1816016427779914
free_energy : 2.7270388301945787

35,33+479
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.35794579207203014
free_energy : 9.213829847470151

35,34+478
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.2016542032849884
free_energy : 4.987704475838655

35+PR_4,62
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.09198700893955447
free_energy : -1.9750776559694714

35+PR_4,63
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.09441645627055095
free_energy : -2.026401738522509

35+PR_5,64
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.26247726442579733
free_energy : -6.570088676106934

35+PR_5,65
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.263785856735808
free_energy : -6.63231131435623

35+

In [25]:
ind=35
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-3.2421475638, left=-3.2421475638, near=-2000.0,…

-1


-7389.618314120459

In [27]:
for neighbor in list(RN.graph.neighbors(122)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

122,123
rxn_type : One electron oxidation
bipartite : 1
energy : 0.1816016427779914
free_energy : 2.7270388301945787

122,119
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : 0.13608182594003893
free_energy : 3.5303255496419297

122+PR_4115,3
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.5128249622669756
free_energy : -13.560052950197132

122+PR_4139,4
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.18390994963908724
free_energy : -4.429906424613364

122+PR_4174,29
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.0505612299340078
free_energy : -0.7976820247390233

122+PR_4175,31
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2965126610809995
free_energy : -7.776907541486449

122+PR_4175,32
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.2892071166939906
free_energy : -7.61447432186034



In [24]:
ind=145
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.10976211932, left=-4.10976211932, near=-2000.…

0


-4166.604920618956

In [48]:
for neighbor in list(RN.graph.neighbors(145)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

145,144
rxn_type : One electron reduction
bipartite : 1
energy : -0.04817687610102439
free_energy : 0.8217637178089716

145+PR_5,265
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.06064451272959559
free_energy : -1.095008192156456

145+PR_48,275
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.046755706438034395
free_energy : -0.6612801349838264

145+PR_90,448
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.07923878441999932
free_energy : -1.5156640899449485

145+PR_151,766
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.03331586886201876
free_energy : -0.4402560319958866

145+PR_611,1706
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.07314353301801191
free_energy : -1.437030540590058

145+PR_768,1957
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.032917536861020835
free_

In [26]:
ind=448
view(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.32244975896, left=-4.32244975896, near=-2000.…

0


-7199.132174674312

In [68]:
from atomate.qchem.database import QChemCalcDb
db_file = "/Users/samuelblau/Desktop/moldb.json"
mmdb = QChemCalcDb.from_db_file(db_file, admin=True)
target_entries = list(mmdb.collection.find({"formula_alphabetical": "C3 H4 Li1 O3", "environment":"smd_18.5,1.415,0.00,0.735,20.2,0.00,0.00", "molecule.charge":0}))
print(len(target_entries),"entries")

42 entries


In [69]:
for entry in target_entries:
    if len(entry["bonds"]) == 11:
        view_old(Molecule.from_dict(entry["molecule"]),bonding_strategy="OpenBabelNN")
        print(entry["free_energy"])

Renderer(background='white', camera=OrthographicCamera(bottom=-6.7520014130399995, left=-6.7520014130399995, n…

-9522.907225166065


Renderer(background='white', camera=OrthographicCamera(bottom=-6.345321399, left=-6.345321399, near=-2000.0, p…

-9522.915866743


Renderer(background='white', camera=OrthographicCamera(bottom=-5.747266307399999, left=-5.747266307399999, nea…

...

-9519.494686091059


Renderer(background='white', camera=OrthographicCamera(bottom=-7.27076544228, left=-7.27076544228, near=-2000.…

-9520.752748758012


Renderer(background='white', camera=OrthographicCamera(bottom=-6.673858932599999, left=-6.673858932599999, nea…

-9518.748204551022


In [73]:
from atomate.qchem.database import QChemCalcDb
db_file = "/Users/samuelblau/Desktop/mol_hetal_db.json"
mmdb = QChemCalcDb.from_db_file(db_file, admin=True)
minus_entries = list(mmdb.collection.find({"formula_alphabetical": "C3 H5 Li1 O4", "environment":"smd_18.5,1.415,0.00,0.735,20.2,0.00,0.00", "molecule.charge":0}))
print(len(minus_entries),"entries")

4 entries


In [74]:
for entry in minus_entries:
    view_old(Molecule.from_dict(entry["molecule"]),bonding_strategy="OpenBabelNN")
    print(entry["free_energy"])

Renderer(background='white', camera=OrthographicCamera(bottom=-6.6942235752, left=-6.6942235752, near=-2000.0,…

-11586.852662876116


Renderer(background='white', camera=OrthographicCamera(bottom=-8.21378872128, left=-8.21378872128, near=-2000.…

-11587.839161760392


Renderer(background='white', camera=OrthographicCamera(bottom=-5.804452683959999, left=-5.804452683959999, nea…

-11586.918897659294


Renderer(background='white', camera=OrthographicCamera(bottom=-6.0211612703999995, left=-6.0211612703999995, n…

-11586.974976056468
